# Inerpretation-Net Training

# Experiment 1: I-Net Performance for Different Algebras and Complexities
# Experiment 2: I-Net Performance Comparison for λ-Nets with Different Training Levels
# Experiment 3: I-Net Performance Comparison Different Training Data Sizes

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'data': {
        'd': 5, #degree
        'n': 1, #number of variables
        'sparsity': None,
        'sample_sparsity': None,
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        'a_max': 1,
        'a_min': -1,
        'lambda_nets_total': 1000,
        'noise': 0.1,
        'noise_distrib': 'normal', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        
        'same_training_all_lambda_nets': False,

        'fixed_seed_lambda_training': True,
        'fixed_initialization_lambda_training': False,
        'number_different_lambda_trainings': 1,
    },
    'lambda_net': {
        'epochs_lambda': 1000,
        'early_stopping_lambda': True,  #if early stopping is used, multi_epoch_analysis is deactivated
        'early_stopping_min_delta_lambda': 1e-4,
        'batch_lambda': 64,
        'dropout': 0,
        'lambda_network_layers': [5*'sparsity'],
        'optimizer_lambda': 'adam',
        'loss_lambda': 'mae',
        'number_of_lambda_weights': None,
        'lambda_dataset_size': 5000,
    },
    'i_net': {
        'optimizer': 'custom',#adam
        'inet_loss': 'mae',
        'inet_metrics': ['r2'],
        'dropout': 0.25,
        'dropout_output': 0,
        'epochs': 500,
        'early_stopping': True,
        'batch_size': 512,
        'dense_layers': [512, 1024],
        'convolution_layers': None,
        'lstm_layers': None,
        'interpretation_dataset_size': 1000,
                
        'interpretation_net_output_monomials': 3, #(None, int)
        'interpretation_net_output_shape': None, #calculated automatically later
        
        'evaluate_with_real_function': False,
        'consider_labels_training': False,
                      
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2)
        'nas': True,
        'nas_type': 'SEQUENTIAL', #options:(None, 'SEQUENTIAL', 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 100,
    },
    'evaluation': {   
        'inet_holdout_seed_evaluation': False,
        
        #set if multi_epoch_analysis should be performed
        'multi_epoch_analysis': True,
        'each_epochs_save_lambda': 20,
        'epoch_start': 0, #use to skip first epochs in multi_epoch_analysis
        
        #set if samples analysis should be performed
        'samples_list': None,#[100, 500, 750, 1000, 2500, 5000, 7500, 10000, 15000, 20000, 25000, 28125] 
       
        'random_evaluation_dataset_size': 500,
    },
    'computation':{
        'n_jobs': 11,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}

## Imports

In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [3]:
#######################################################################################################################################
##################################################### IMPORT LIBRARIES ################################################################
#######################################################################################################################################
from itertools import product       
from tqdm import tqdm_notebook as tqdm
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import psutil

from functools import reduce
from more_itertools import random_product 
from sklearn.preprocessing import Normalizer

import sys
import os
import shutil

import logging

from prettytable import PrettyTable
import colored
import math

import time
from datetime import datetime
from collections.abc import Iterable


from joblib import Parallel, delayed

from scipy.integrate import quad

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score
from similaritymeasures import frechet_dist, area_between_two_curves, dtw
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils import plot_model
from IPython.display import Image

import keras.backend as K
from livelossplot import PlotLossesKerasTF
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from matplotlib import pyplot as plt
import seaborn as sns


import tensorflow as tf
import random 


import warnings


from IPython.display import display, Math, Latex



In [4]:
sys.path.append('..')

from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *

In [5]:
#######################################################################################################################################
################################################### VARIABLE ADJUSTMENTS ##############################################################
#######################################################################################################################################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

n_jobs = min((epochs_lambda//each_epochs_save_lambda+1, n_jobs)) if multi_epoch_analysis else min(len(samples_list), n_jobs) if samples_list!=None else 1

multi_epoch_analysis = False if early_stopping_lambda else multi_epoch_analysis #deactivate multi_epoch_analysis if early stopping is used

each_epochs_save_lambda = each_epochs_save_lambda if multi_epoch_analysis else epochs_lambda
epochs_save_range_lambda = range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda) if each_epochs_save_lambda == 1 else range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda+1) if multi_epoch_analysis else range(1,2)

data_reshape_version = 2 if data_reshape_version == None and (convolution_layers != None or lstm_layers != None or (nas and nas_type != 'SEQUENTIAL')) else data_reshape_version
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")
#np.set_printoptions(suppress=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [6]:
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################
config['evaluation']['multi_epoch_analysis'] = multi_epoch_analysis
config['evaluation']['each_epochs_save_lambda'] = each_epochs_save_lambda
config['i_net']['data_reshape_version'] = data_reshape_version

config['data']['sparsity'] = nCr(config['data']['n']+config['data']['d'], config['data']['d'])
config['data']['sample_sparsity'] = config['data']['sparsity'] if config['data']['sample_sparsity'] == None else config['data']['sample_sparsity']

config['i_net']['interpretation_net_output_shape'] = config['data']['sparsity'] if config['i_net']['interpretation_net_output_monomials'] is None else config['data']['sparsity']*config['i_net']['interpretation_net_output_monomials']+config['i_net']['interpretation_net_output_monomials']


transformed_layers = []
for layer in config['lambda_net']['lambda_network_layers']:
    if type(layer) == str:
        transformed_layers.append(layer.count('sparsity')*config['data']['sparsity'])
    else:
        transformed_layers.append(layer)
config['lambda_net']['lambda_network_layers'] = transformed_layers

layers_with_input_output = list(flatten([[config['data']['n']], config['lambda_net']['lambda_network_layers'], [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]  
config['lambda_net']['number_of_lambda_weights'] = number_of_lambda_weights
    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])



initialize_LambdaNet_config_from_curent_notebook(config)
initialize_metrics_config_from_curent_notebook(config)
initialize_utility_functions_config_from_curent_notebook(config)
initialize_InterpretationNet_config_from_curent_notebook(config)


#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='interpretation_net'))
create_folders_inet()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [7]:
print(path_identifier_interpretation_net_data)

print(path_identifier_lambda_net_data)


inet_dense512-1024-output_21_drop0.25e500b512_custom/lnets_1000_30-1000e_ES0.0001_64b_adam_mae_train_5000_diffX_1-FixSeed_42/var_1_d_5_sparsity_6_amin_-1_amax_1_xdist_uniform_noise_normal_0.1
lnets_1000_30-1000e_ES0.0001_64b_adam_mae_train_5000_diffX_1-FixSeed_42/var_1_d_5_sparsity_6_amin_-1_amax_1_xmin_0_xmax_1_xdist_uniform_noise_normal_0.1


In [8]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


## Utility functions

### Generate List of Monomial Identifiers

In [9]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)


layers_with_input_output = list(flatten([[n], lambda_network_layers, [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]

  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


## Load Data and Generate Datasets

In [10]:
def load_lambda_nets(index):
    
    if psutil.virtual_memory().percent > 80:
        raise SystemExit("Out of RAM!")
    
    directory = './data/weights/' + 'weights_' + path_identifier_lambda_net_data + '/'
    path_weights = directory + 'weights_epoch_' + str(index).zfill(3) + '.txt'
    path_X_data = directory + 'lambda_X_test_data.txt'
    path_y_data = directory + 'lambda_y_test_data.txt'        
    
    weight_data = pd.read_csv(path_weights, sep=",", header=None)
    weight_data = weight_data.sort_values(by=0).sample(frac=1, random_state=RANDOM_SEED)
    weight_data = weight_data.sort_values(by=0).sample(n=interpretation_dataset_size, random_state=RANDOM_SEED)
    
    lambda_X_test_data = pd.read_csv(path_X_data, sep=",", header=None)
    lambda_X_test_data = lambda_X_test_data.sort_values(by=0).sample(frac=1, random_state=RANDOM_SEED)
    lambda_X_test_data = lambda_X_test_data.sort_values(by=0).sample(n=interpretation_dataset_size, random_state=RANDOM_SEED)
    
    lambda_y_test_data = pd.read_csv(path_y_data, sep=",", header=None)
    lambda_y_test_data = lambda_y_test_data.sort_values(by=0).sample(frac=1, random_state=RANDOM_SEED)
    lambda_y_test_data = lambda_y_test_data.sort_values(by=0).sample(n=interpretation_dataset_size, random_state=RANDOM_SEED)
        
    lambda_nets = [None] * weight_data.shape[0]
    for i, (row_weights, row_lambda_X_test_data, row_lambda_y_test_data) in enumerate(zip(weight_data.values, lambda_X_test_data.values, lambda_y_test_data.values)):
        lambda_net = LambdaNet(row_weights, row_lambda_X_test_data, row_lambda_y_test_data)
        lambda_nets[i] = lambda_net
                
    lambda_net_dataset = LambdaNetDataset(lambda_nets)
        
    return lambda_net_dataset
    

In [11]:
#LOAD DATA

parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
lambda_net_dataset_list = parallel(delayed(load_lambda_nets)((i+1)*each_epochs_save_lambda if each_epochs_save_lambda==1 else i*each_epochs_save_lambda if i > 1 else each_epochs_save_lambda if i==1 else 1) for i in epochs_save_range_lambda)  
del parallel

lambda_net_dataset = lambda_net_dataset_list[-1]


[Parallel(n_jobs=11)]: Using backend MultiprocessingBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   1 out of   1 | elapsed:    9.1s finished


## Data Inspection

In [12]:
lambda_net_dataset.as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
521,1373158606,0.346,-0.838,0.903,0.677,0.610,0.646,0.300,-0.445,0.835,-3.087,7.851,-3.143,0.300,-0.445,0.835,-3.087,7.851,-3.143,-0.009,-0.227,0.415,0.079,0.717,0.594,0.526,-0.297,0.066,1.173,0.444,-0.088,-0.019,0.456,0.610,0.552,-0.043,0.823,0.271,0.814,0.726,0.041,0.008,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.005,-0.082,-0.233,-0.348,-0.463,0.000,-0.069,-0.817,-0.123,0.000,0.000,-0.404,-0.511,-0.006,0.136,-0.482,0.153,-0.369,-0.577,-0.047,-0.023,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.403,-0.222,0.855,0.976,1.034,-0.408,-0.257,1.209,0.254,-0.352,-0.364,0.785,1.189,-0.358,0.566,0.918,0.283,0.850,1.445,-0.310,-0.044,-0.261,-0.383,-0.059,-0.334,0.144,-0.258,-0.315,0.171
737,1373158606,0.199,0.457,-0.369,-0.214,-0.526,-0.806,0.233,0.021,1.036,-0.867,-2.511,0.865,0.233,0.021,1.036,-0.867,-2.511,0.865,-0.009,-0.227,0.534,0.434,0.187,0.119,0.524,-0.297,0.409,0.649,0.673,-0.088,-0.019,0.569,0.114,0.661,-0.023,0.241,0.277,0.258,-0.011,0.379,0.416,-0.042,-0.404,-0.090,0.393,-0.291,-0.209,-0.233,0.000,0.000,-0.313,-0.347,0.078,0.081,-0.363,0.000,-0.351,-0.445,-0.308,0.000,0.000,-0.333,0.080,-0.272,0.095,0.064,0.078,0.071,0.094,-0.324,-0.351,0.000,0.000,0.000,-0.315,0.000,0.000,0.000,-0.254,-0.359,-0.885,-1.076,0.206,0.249,-0.845,-0.408,-1.190,-0.741,-0.668,-0.352,-0.364,-0.779,0.185,-0.736,0.499,0.082,0.235,0.120,0.127,-1.323,-1.202,-0.261,-0.383,-0.059,-1.320,0.144,-0.258,-0.315,0.085
740,1373158606,0.749,0.661,-0.551,-0.201,-0.181,0.956,0.724,0.694,-1.227,2.608,-2.511,0.897,0.724,0.694,-1.227,2.608,-2.511,0.897,-0.009,-0.227,0.264,0.083,0.295,0.232,0.225,-0.297,0.067,0.296,0.375,-0.088,-0.019,0.266,0.228,0.362,0.090,0.344,0.382,0.360,0.131,0.041,0.008,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.017,-0.083,0.189,0.188,-0.010,0.000,-0.069,0.197,-0.035,0.000,0.000,-0.017,0.193,-0.033,0.184,0.203,0.185,0.196,0.216,-0.047,-0.023,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.275,-0.226,0.349,0.389,-0.118,-0.408,-0.258,0.096,-0.130,-0.352,-0.364,-0.131,0.325,-0.209,0.604,0.227,0.377,0.263,0.258,-0.310,-0.044,-0.261,-0.383,-0.059,-0.335,0.144,-0.258,-0.315,0.167
660,1373158606,0.148,-0.214,-0.943,0.167,-0.977,0.575,0.126,0.208,-3.691,7.827,-10.374,4.704,0.126,0.208,-3.691,7.827,-10.374,4.704,-0.009,-0.227,0.484,0.364,0.091,0.069,0.445,-0.297,0.328,0.555,0.636,-0.088,-0.019,0.501,0.071,0.611,0.008,0.246,0.172,0.199,0.033,0.284,0.299,-0.042,-0.404,-0.090,0.290,-0.291,-0.209,-0.233,0.000,0.000,-0.115,-0.190,0.072,-0.079,-0.189,0.000,-0.195,-0.231,-0.051,0.000,0.000,-0.128,-0.080,-0.047,-0.025,-0.001,0.083,0.044,-0.045,-0.182,-0.194,0.000,0.000,0.000,-0.186,0.000,0.000,0.000,-0.254,-0.359,-0.514,-0.560,0.091,0.165,-0.382,-0.408,-0.601,-0.269,-0.372,-0.352,-0.364,-0.375,0.102,-0.449,0.393,-0.060,0.119,-0.007,0.047,-0.674,-0.491,-0.261,-0.383,-0.059,-0.692,0.144,-0.258,-0.315,0.111
411,1373158606,-0.210,-0.921,-0.049,0.086,-0.545,0.928,-0.203,-1.317,3.256,-9.844,11.653,-4.296,-0.203,-1.317,3.256,-9.844,11.653,-4.296,-0.009,-0.227,0.364,0.258,0.345,0.317,0.320,-0.297,0.225,0.383,0.488,-0.088,-0.019,0.367,0.334,0.473,0.008,0.399,0.406,0.420,0.034,0.173,0.111,-0.042,-0.404,-0.090,0.185,-0.291,-0.209,-0.233,0.000,0

In [13]:
lambda_net_dataset.as_pandas().describe()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
count,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000
mean,1373158606.000,0.004,-0.027,-0.039,0.029,-0.001,-0.001,0.007,-0.056,0.194,-0.844,1.212,-0.544,0.007,-0.056,0.194,-0.844,1.212,-0.544,-0.009,-0.227,0.290,0.194,0.277,0.217,0.282,-0.297,0.166,0.449,0.474,-0.088,-0.019,0.325,0.219,0.406,0.073,0.408,0.362,0.388,0.144,0.114,0.062,-0.042,-0.404,-0.090,0.125,-0.291,-0.209,-0.233,0.000,0.000,-0.013,-0.026,-0.007,-0.014,-0.052,0.000,-0.025,-0.073,-0.041,0.000,0.000,-0.044,-0.021,-0.000,0.017,-0.063,0.020,-0.030,-0.012,-0.016,0.003,0.000,0.000,0.000,-0.017,0.000,0.000,0.000,-0.254,-0.359,-0.372,-0.422,0.352,0.418,-0.201,-0.408,-0.466,0.028,-0.115,-0.352,-0.364,-0.187,0.363,-0.271,0.615,0.123,0.353,0.174,0.325,-0.519,-0.263,-0.261,-0.383,-0.059,-0.543,0.144,-0.258,-0.315,0.008
std,0.000,0.575,0.588,0.580,0.569,0.582,0.579,0.542,0.660,3.414,10.219,13.364,5.553,0.542,0.660,3.414,10.219,13.364,5.553,0.000,0.000,0.142,0.141,0.157,0.154,0.178,0.000,0.141,0.275,0.195,0.000,0.000,0.166,0.161,0.142,0.145,0.234,0.159,0.184,0.200,0.140,0.184,0.000,0.000,0.000,0.139,0.000,0.000,0.000,0.000,0.000,0.143,0.157,0.170,0.179,0.193,0.000,0.158,0.290,0.211,0.000,0.000,0.185,0.191,0.138,0.151,0.246,0.139,0.213,0.210,0.152,0.179,0.000,0.000,0.000,0.151,0.000,0.000,0.000,0.000,0.000,0.169,0.227,0.213,0.250,0.367,0.000,0.257,0.436,0.436,0.000,0.000,0.348,0.265,0.216,0.301,0.483,0.174,0.429,0.349,0.282,0.336,0.000,0.000,0.000,0.271,0.000,0.000,0.000,0.141
min,1373158606.000,-1.000,-0.998,-0.999,-1.000,-1.000,-1.000,-1.007,-2.548,-10.409,-73.162,-54.616,-41.140,-1.007,-2.548,-10.409,-73.162,-54.616,-41.140,-0.009,-0.227,-0.128,-0.216,-0.145,-0.231,-0.146,-0.297,-0.244,-0.109,0.145,-0.088,-0.019,-0.100,-0.242,0.067,-0.301,-0.049,-0.012,-0.019,-0.351,-0.286,-0.633,-0.042,-0.404,-0.090,-0.273,-0.291,-0.209,-0.233,0.000,0.000,-0.389,-0.444,-0.495,-0.533,-0.952,0.000,-0.495,-1.097,-1.180,0.000,0.000,-0.866,-0.539,-0.647,-0.466,-0.933,-0.341,-1.001,-0.704,-0.476,-0.812,0.000,0.000,0.000,-0.476,0.000,0.000,0.000,-0.254,-0.359,-1.131,-1.399,0.001,0.092,-1.147,-0.408,-2.213,-2.000,-0.918,-0.352,-0.364,-1.072,0.025,-0.973,0.317,-2.223,0.024,-2.786,-0.022,-2.082,-2.198,-0.261,-0.383,-0.059,-1.868,0.144,-0.258,-0.315,-0.498
25%,1373158606.000,-0.497,-0.527,-0.550,-0.456,-0.516,-0.4

## Generate Datasets for Interpretation-Net training

In [14]:
#generate train, test and validation data for training

lambda_net_train_dataset_list = []
lambda_net_valid_dataset_list = []
lambda_net_test_dataset_list = []

for lambda_net_dataset in lambda_net_dataset_list:
    
    
    if inet_holdout_seed_evaluation:
        complete_seed_list = list(set(lambda_net_dataset.train_settings_list['seed']))#list(weight_data.iloc[:,1].unique())

        random.seed(RANDOM_SEED)
        test_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_test_dataset = lambda_net_dataset.get_lambda_nets_by_seed(test_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(test_seeds))#complete_seed_list.remove(test_seeds)
        
        random.seed(RANDOM_SEED)
        valid_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_valid_dataset = lambda_net_dataset.get_lambda_nets_by_seed(valid_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(valid_seeds))

        train_seeds = complete_seed_list
        lambda_net_train_dataset = lambda_net_dataset.get_lambda_nets_by_seed(train_seeds)       
        
        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
        
        del lambda_net_dataset
    else:
        lambda_net_train_with_valid_dataset, lambda_net_test_dataset = split_LambdaNetDataset(lambda_net_dataset, test_split=0.1)
        lambda_net_train_dataset, lambda_net_valid_dataset = split_LambdaNetDataset(lambda_net_train_with_valid_dataset, test_split=0.1)

        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
    
        del lambda_net_dataset, lambda_net_train_with_valid_dataset

        
del lambda_net_dataset_list

In [15]:
lambda_net_train_dataset_list[-1].as_pandas().shape

(810, 110)

In [16]:
lambda_net_valid_dataset_list[-1].as_pandas().shape

(90, 110)

In [17]:
lambda_net_test_dataset_list[-1].as_pandas().shape

(100, 110)

In [18]:
lambda_net_train_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
718,1373158606,0.657,0.567,-0.446,0.882,-0.751,0.750,0.660,0.066,4.295,-13.973,17.875,-7.326,0.660,0.066,4.295,-13.973,17.875,-7.326,-0.009,-0.227,0.212,0.083,0.294,0.231,0.663,-0.297,0.067,0.309,0.356,-0.088,-0.019,0.536,0.228,0.350,0.089,0.345,0.380,0.360,0.138,0.041,0.008,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.006,-0.083,0.174,0.174,-0.422,0.000,-0.069,0.154,-0.010,0.000,0.000,-0.406,0.177,-0.009,0.171,0.183,0.172,0.179,0.190,-0.047,-0.022,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.179,-0.226,0.337,0.375,1.223,-0.408,-0.258,0.084,-0.095,-0.352,-0.364,0.894,0.310,-0.177,0.578,0.216,0.367,0.253,0.235,-0.310,-0.044,-0.261,-0.383,-0.059,-0.335,0.144,-0.258,-0.315,0.162
457,1373158606,-0.608,-0.718,0.245,0.563,0.157,-0.706,-0.610,-0.872,1.170,-0.605,-0.544,0.465,-0.610,-0.872,1.170,-0.605,-0.544,0.465,-0.009,-0.227,0.289,0.181,0.252,0.199,0.240,-0.297,0.148,0.292,0.412,-0.088,-0.019,0.289,0.198,0.398,0.008,0.300,0.329,0.313,0.033,0.095,0.007,-0.042,-0.404,-0.090,0.108,-0.291,-0.209,-0.233,0.000,0.000,0.142,0.144,-0.057,-0.045,0.151,0.000,0.144,0.177,0.146,0.000,0.000,0.148,-0.046,0.143,-0.023,-0.066,-0.070,-0.068,-0.039,0.144,0.181,0.000,0.000,0.000,0.143,0.000,0.000,0.000,-0.254,-0.359,-0.353,-0.365,0.254,0.297,-0.201,-0.408,-0.397,-0.088,-0.209,-0.352,-0.364,-0.212,0.232,-0.286,0.393,0.129,0.284,0.167,0.048,-0.454,-0.226,-0.261,-0.383,-0.059,-0.476,0.144,-0.258,-0.315,-0.134
847,1373158606,0.675,-0.520,-0.610,0.010,0.537,-0.876,0.657,0.053,-4.872,12.507,-15.009,5.953,0.657,0.053,-4.872,12.507,-15.009,5.953,-0.009,-0.227,0.473,0.344,0.049,-0.018,0.406,-0.297,0.312,0.527,0.624,-0.088,-0.019,0.474,-0.027,0.601,-0.161,0.094,0.144,0.116,-0.148,0.278,0.307,-0.042,-0.404,-0.090,0.282,-0.291,-0.209,-0.233,0.000,0.000,-0.091,-0.201,0.176,0.181,-0.217,0.000,-0.219,-0.325,-0.016,0.000,0.000,-0.143,0.186,-0.015,0.187,0.175,0.168,0.171,0.221,-0.215,-0.220,0.000,0.000,0.000,-0.215,0.000,0.000,0.000,-0.254,-0.359,-0.509,-0.604,0.301,0.389,-0.402,-0.408,-0.685,-0.398,-0.371,-0.352,-0.364,-0.375,0.330,-0.448,0.712,0.148,0.278,0.174,0.346,-0.810,-0.658,-0.261,-0.383,-0.059,-0.814,0.144,-0.258,-0.315,0.168
921,1373158606,0.278,-0.837,-0.130,-0.415,0.091,-0.733,0.257,-0.239,-4.418,11.585,-14.241,5.341,0.257,-0.239,-4.418,11.585,-14.241,5.341,-0.009,-0.227,0.568,0.464,0.077,0.081,0.543,-0.297,0.440,0.670,0.725,-0.088,-0.019,0.592,0.080,0.695,0.008,0.333,0.126,0.213,0.034,0.392,0.443,-0.042,-0.404,-0.090,0.395,-0.291,-0.209,-0.233,0.000,0.000,-0.082,-0.310,-0.015,-0.089,-0.321,0.000,-0.315,-0.352,-0.003,0.000,0.000,-0.235,-0.087,-0.001,-0.026,0.000,0.177,-0.001,-0.045,-0.317,-0.357,0.000,0.000,0.000,-0.318,0.000,0.000,0.000,-0.254,-0.359,-0.589,-0.827,0.026,0.179,-0.604,-0.408,-0.909,-0.465,-0.449,-0.352,-0.364,-0.529,0.113,-0.526,0.392,-0.135,0.149,-0.039,0.048,-1.043,-0.949,-0.261,-0.383,-0.059,-1.046,0.144,-0.258,-0.315,0.241
938,1373158606,0.093,-0.817,0.513,0.165,-0.627,-0.624,0.074,-0.115,-4.506,13.784,-16.501,6.020,0.074,-0.115,-4.506,13.784,-16.501,6.020,-0.009,-0.227,0.433,0.408,0.121,0.069,0.430,-0.297,0.383,0.567,0.567,-0.088,-0.019,0.443,0.072,0.546,0.008,0.202,0.205,0.197,0.033,0.364,0.422,-0.042,-0.404,-0.090,0.351,-0.291,-0.209,-0.2

In [19]:
lambda_net_valid_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
580,1373158606,-0.693,0.145,-0.446,0.843,0.166,0.186,-0.687,0.050,0.689,-3.641,6.891,-3.163,-0.687,0.050,0.689,-3.641,6.891,-3.163,-0.009,-0.227,0.104,-0.015,0.420,0.363,0.046,-0.297,-0.051,0.091,0.241,-0.088,-0.019,0.103,0.374,0.222,0.271,0.487,0.505,0.507,0.355,-0.108,-0.220,-0.042,-0.404,-0.090,-0.093,-0.291,-0.209,-0.233,0.000,0.000,0.152,0.159,-0.249,-0.256,0.160,0.000,0.161,0.170,0.146,0.000,0.000,0.154,-0.271,0.148,-0.220,-0.299,-0.210,-0.245,-0.288,0.165,0.221,0.000,0.000,0.000,0.163,0.000,0.000,0.000,-0.254,-0.359,-0.293,-0.372,0.593,0.701,-0.168,-0.408,-0.433,-0.038,-0.107,-0.352,-0.364,-0.153,0.657,-0.188,1.094,0.483,0.544,0.477,0.716,-0.543,-0.380,-0.261,-0.383,-0.059,-0.552,0.144,-0.258,-0.315,-0.151
451,1373158606,-0.760,-0.469,-0.965,-0.413,0.546,0.036,-0.707,-1.319,3.445,-10.673,11.375,-4.154,-0.707,-1.319,3.445,-10.673,11.375,-4.154,-0.009,-0.227,0.339,0.232,0.114,0.081,0.298,-0.297,0.199,0.375,0.465,-0.088,-0.019,0.344,0.079,0.448,0.008,0.599,0.145,0.480,0.033,0.147,0.095,-0.042,-0.404,-0.090,0.158,-0.291,-0.209,-0.233,0.000,0.000,0.154,0.155,-0.107,-0.081,0.161,0.000,0.154,0.180,0.158,0.000,0.000,0.159,-0.079,0.155,-0.023,-0.235,-0.136,-0.174,-0.039,0.154,0.173,0.000,0.000,0.000,0.153,0.000,0.000,0.000,-0.254,-0.359,-0.395,-0.400,0.140,0.187,-0.240,-0.408,-0.428,-0.127,-0.253,-0.352,-0.364,-0.253,0.121,-0.330,0.393,-0.601,0.131,-0.460,0.048,-0.476,-0.215,-0.261,-0.383,-0.059,-0.501,0.144,-0.258,-0.315,-0.146
530,1373158606,-0.779,-0.422,-0.380,-0.500,0.030,0.071,-0.720,-1.058,1.707,-2.550,-0.597,1.308,-0.720,-1.058,1.707,-2.550,-0.597,1.308,-0.009,-0.227,0.317,0.210,0.107,0.081,0.273,-0.297,0.177,0.346,0.442,-0.088,-0.019,0.320,0.079,0.426,0.008,0.750,0.303,0.463,0.033,0.124,0.065,-0.042,-0.404,-0.090,0.136,-0.291,-0.209,-0.233,0.000,0.000,0.160,0.161,-0.107,-0.081,0.167,0.000,0.160,0.188,0.164,0.000,0.000,0.166,-0.079,0.161,-0.023,-0.369,-0.004,-0.277,-0.040,0.160,0.182,0.000,0.000,0.000,0.159,0.000,0.000,0.000,-0.254,-0.359,-0.384,-0.392,0.136,0.188,-0.230,-0.408,-0.421,-0.117,-0.241,-0.352,-0.364,-0.243,0.122,-0.318,0.394,-0.869,0.222,-0.500,0.049,-0.472,-0.216,-0.261,-0.383,-0.059,-0.497,0.144,-0.258,-0.315,-0.151
894,1373158606,0.030,0.981,0.223,-0.258,0.544,-0.659,0.011,1.352,-2.394,7.357,-8.982,3.574,0.011,1.352,-2.394,7.357,-8.982,3.574,-0.009,-0.227,0.250,0.207,0.400,0.336,0.234,-0.297,0.200,0.507,0.285,-0.088,-0.019,0.250,0.337,0.299,0.189,0.463,0.484,0.473,0.260,0.041,0.009,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.171,-0.146,0.002,0.003,-0.165,0.000,-0.139,-0.003,-0.194,0.000,0.000,-0.171,-0.000,-0.201,0.007,-0.003,0.003,-0.003,-0.002,-0.047,-0.022,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.386,-0.491,0.363,0.390,-0.246,-0.408,-0.578,0.149,-0.134,-0.352,-0.364,-0.224,0.325,-0.257,0.548,0.248,0.403,0.286,0.227,-0.310,-0.045,-0.261,-0.383,-0.059,-0.335,0.144,-0.258,-0.315,0.014
590,1373158606,-0.579,0.618,-0.706,-0.315,0.729,-0.690,-0.409,-0.291,0.109,-0.247,0.247,-0.091,-0.409,-0.291,0.109,-0.247,0.247,-0.091,-0.009,-0.227,0.285,0.178,0.245,0.186,0.238,-0.297,0.145,0.294,0.408,-0.088,-0.019,0.286,0.182,0.393,0.008,0.293,0.329,0.309,0.034,0.092,0.014,-0.042,-0.404,-0.090,0.105,-0.291,-0.209,-0.233,0.

In [20]:
lambda_net_test_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
666,1373158606,-0.331,-0.211,0.060,-0.677,0.144,0.611,-0.306,-0.680,2.165,-4.971,4.472,-1.100,-0.306,-0.680,2.165,-4.971,4.472,-1.100,-0.009,-0.227,0.253,0.146,0.291,0.270,0.204,-0.297,0.114,0.236,0.374,-0.088,-0.019,0.252,0.280,0.362,0.009,0.283,0.315,0.293,0.287,0.061,-0.026,-0.042,-0.404,-0.090,0.074,-0.291,-0.209,-0.233,0.000,0.000,0.088,0.089,-0.224,-0.210,0.087,0.000,0.089,0.071,0.086,0.000,0.000,0.087,-0.216,0.087,-0.023,-0.218,-0.039,-0.047,-0.223,0.090,0.104,0.000,0.000,0.000,0.090,0.000,0.000,0.000,-0.254,-0.359,-0.291,-0.294,0.521,0.670,-0.134,-0.408,-0.322,-0.022,-0.152,-0.352,-0.364,-0.149,0.628,-0.229,0.394,0.253,0.239,0.117,0.910,-0.375,-0.143,-0.261,-0.383,-0.059,-0.398,0.144,-0.258,-0.315,-0.089
129,1373158606,0.595,-0.700,-0.541,0.445,0.440,0.282,0.485,-0.349,-0.065,0.148,-0.149,0.055,0.485,-0.349,-0.065,0.148,-0.149,0.055,-0.009,-0.227,0.318,0.216,0.213,0.149,0.278,-0.297,0.185,0.344,0.441,-0.088,-0.019,0.323,0.144,0.425,0.005,0.258,0.301,0.277,0.031,0.136,0.008,-0.042,-0.404,-0.090,0.148,-0.291,-0.209,-0.233,0.000,0.000,-0.003,-0.002,0.144,0.145,-0.002,0.000,-0.001,-0.003,-0.005,0.000,0.000,-0.003,0.149,-0.005,0.151,0.150,0.141,0.146,0.182,-0.001,-0.023,0.000,0.000,0.000,-0.001,0.000,0.000,0.000,-0.254,-0.359,-0.331,-0.330,0.271,0.318,-0.172,-0.408,-0.356,-0.053,-0.195,-0.352,-0.364,-0.189,0.255,-0.272,0.598,0.146,0.294,0.181,0.227,-0.402,-0.045,-0.261,-0.383,-0.059,-0.429,0.144,-0.258,-0.315,0.135
784,1373158606,-0.170,-0.156,0.570,-0.609,0.641,0.297,-0.179,-0.053,0.327,-1.006,2.125,-0.680,-0.179,-0.053,0.327,-1.006,2.125,-0.680,-0.009,-0.227,0.119,0.008,0.396,0.365,0.110,-0.297,-0.028,0.468,0.273,-0.088,-0.019,0.150,0.354,0.236,0.252,0.461,0.500,0.497,0.322,-0.077,-0.003,-0.042,-0.404,-0.090,-0.071,-0.291,-0.209,-0.233,0.000,0.000,0.070,0.071,-0.282,-0.259,0.034,0.000,0.073,-0.332,0.044,0.000,0.000,0.045,-0.282,0.065,-0.207,-0.274,-0.193,-0.211,-0.263,0.083,-0.009,0.000,0.000,0.000,0.078,0.000,0.000,0.000,-0.254,-0.359,-0.158,-0.182,0.590,0.691,-0.003,-0.408,-0.227,0.363,-0.009,-0.352,-0.364,-0.013,0.638,-0.087,1.084,0.420,0.510,0.426,0.685,-0.337,-0.034,-0.261,-0.383,-0.059,-0.343,0.144,-0.258,-0.315,-0.080
152,1373158606,0.248,-0.181,0.104,-0.128,-0.411,0.897,0.249,-0.618,4.411,-13.944,17.458,-7.119,0.249,-0.618,4.411,-13.944,17.458,-7.119,-0.009,-0.227,0.286,0.181,0.232,0.170,0.238,-0.297,0.149,0.868,0.406,-0.088,-0.019,0.285,0.166,0.393,0.030,0.278,0.319,0.296,0.068,0.036,0.008,-0.042,-0.404,-0.090,0.109,-0.291,-0.209,-0.233,0.000,0.000,-0.000,0.000,0.082,0.082,-0.000,0.000,0.000,-0.587,-0.001,0.000,0.000,-0.001,0.082,-0.001,0.083,0.081,0.082,0.081,0.083,-0.041,-0.023,0.000,0.000,0.000,-0.005,0.000,0.000,0.000,-0.254,-0.359,-0.288,-0.275,0.238,0.271,-0.125,-0.408,-0.294,1.133,-0.158,-0.352,-0.364,-0.146,0.206,-0.235,0.467,0.119,0.274,0.157,0.122,-0.304,-0.044,-0.261,-0.383,-0.059,-0.348,0.144,-0.258,-0.315,0.082
920,1373158606,0.669,0.155,0.293,-0.083,0.057,-0.931,0.649,0.734,-4.022,12.154,-14.618,5.309,0.649,0.734,-4.022,12.154,-14.618,5.309,-0.009,-0.227,0.383,0.333,0.170,0.105,0.401,-0.297,0.311,0.087,0.512,-0.088,-0.019,0.422,0.094,0.452,-0.040,0.208,0.264,0.232,-0.042,0.323,0.008,-0.042,-0.404,-0.090,0.304,-0.291,-0.209,-0.233,0.000,0.

## Interpretation Network Training

In [ ]:
(history_list, 
scores_list, 

function_values_complete_list, 
function_values_valid_list, 
function_values_test_list, 

inet_preds_list, 
inet_preds_valid_list, 
inet_preds_test_list, 

distrib_dict_list,
model_list) = calculate_interpretation_net_results(lambda_net_train_dataset_list, 
                                                   lambda_net_valid_dataset_list, 
                                                   lambda_net_test_dataset_list)

Trial 2 Complete [00h 00m 35s]
val_loss: 0.2338481843471527

Best val_loss So Far: 0.18888302147388458
Total elapsed time: 00h 01m 18s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
dense_block_1/u...|False             |False             
dense_block_1/n...|2                 |2                 
dense_block_1/u...|512               |512               
dense_block_1/d...|0                 |0                 
dense_block_1/u...|32                |32                
optimizer         |sgd               |adam              
learning_rate     |0.001             |0.001             

Epoch 1/500
26/26 [==============================] - 10s 386ms/step - loss: 0.6254 - val_loss: 0.6536
Epoch 2/500
26/26 [==============================] - 1s 50ms/step - loss: 0.6189 - val_loss: 0.6473
Epoch 3/500
26/26 [==============================] - 1s 33ms/step - loss: 0.6128 - val_loss: 0.6409
Epoch 4/500
26/26 [==============================] - 2s 80ms/step - loss: 0.606

## Evaluate Interpretation Net

In [ ]:
if nas:
    for trial in history_list[-1]: 
        print(trial.summary())

In [ ]:
if len(model_list) >= 1:
    print(model_list[-1].summary())
    print(model_list[-1].get_config())

In [ ]:
scores_list[-1]

In [ ]:
distrib_dict_list[-1]['MAE']

In [ ]:
distrib_dict_list[-1]['R2']

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_min]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_min])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_min])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_lambda, force_complete_poly_representation=True)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_max]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_max])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_max])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_lambda, force_complete_poly_representation=True)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True)

In [ ]:
r2_values_inet = distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_inet)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_inet = r2_values_inet[r2_values_inet>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_inet)) + ' (' + str(r2_values_positive_inet.shape[0]) + ' Samples)')



In [ ]:
r2_values_lstsq_lambda = distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_lstsq_lambda)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_lstsq_lambda = r2_values_lstsq_lambda[r2_values_lstsq_lambda>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_lstsq_lambda)) + ' (' + str(r2_values_positive_lstsq_lambda.shape[0]) + ' Samples)')



In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50])


In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50])


In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50], binwidth=0.2)
p.set(xlim=(-30, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50], binwidth=0.2)
p.set(xlim=(-10, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history[list(history.keys())[1]])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history[list(history.keys())[len(history.keys())//2+1]]) 
    plt.title('model ' + list(history.keys())[1])
    plt.ylabel('metric')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/metric_' + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')


In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history['loss'])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/loss_' + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')    


### Multi Epoch/Sampes Analysis

### Generate Comparison Plots

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['R2 FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list, ylim=(-5, 1))

#### Generate and Analyze Predictions for Random Function

In [ ]:
index = 6

polynomial_target = lambda_net_test_dataset.target_polynomial_list[index]
polynomial_lstsq_target = lambda_net_test_dataset.lstsq_target_polynomial_list[index]
polynomial_lstsq_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index]
polynomial_inet = inet_preds_test_list[-1][index]

print('Target Poly:')
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True, round_digits=4)
print('LSTSQ Target Poly:')
print_polynomial_from_coefficients(polynomial_lstsq_target, force_complete_poly_representation=True, round_digits=4)
print('LSTSQ Lambda Poly:')
print_polynomial_from_coefficients(polynomial_lstsq_lambda, force_complete_poly_representation=True, round_digits=4)
print('I-Net Poly:')
print_polynomial_from_coefficients(polynomial_inet, round_digits=4)


In [ ]:
lambda_net_test_dataset_list[0].X_test_data_list[0][:10]

In [ ]:
lambda_net_test_dataset_list[0].y_test_data_list[0][:10]

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=1)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=2)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=3)

# BENCHMARK (RANDOM GUESS) EVALUATION

In [ ]:
list_of_random_polynomials = np.random.uniform(low=-10, high=10, size=(len(lambda_net_test_dataset_list[-1]), sparsity))

In [ ]:
true_fv_test = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].X_test_data_list, force_complete_poly_representation=True)
random_fv_test = parallel_fv_calculation_from_polynomial(list_of_random_polynomials, lambda_net_test_dataset_list[-1].X_test_data_list, force_complete_poly_representation=True)

In [ ]:
print('Random Guess Error Coefficients: ' + str(np.round(mean_absolute_error(lambda_net_test_dataset_list[-1].target_polynomial_list, list_of_random_polynomials), 4)))

In [ ]:
print('Random Guess Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, random_fv_test), 4)))

# BENCHMARK (EDUCATED GUESS/MEAN PREDICTION) EVALUATION

In [ ]:
true_fv_train = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].X_test_data_list, force_complete_poly_representation=True)

mean_fv = np.mean(true_fv_train)
mean_fv_pred_test = [mean_fv for _ in range(true_fv_test.shape[0])]

In [ ]:
print('Educated Guess/Mean Prediction Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, mean_fv_pred_test), 4)))

In [ ]:
%%script false --no-raise-error

base_model = generate_base_model()
random_evaluation_dataset = np.random.uniform(low=x_min, high=x_max, size=(random_evaluation_dataset_size, n))
#random_evaluation_dataset = lambda_train_input_train_split[0]#lambda_train_input[0] #JUST [0] HERE BECAUSE EVALUATION ALWAYS ON THE SAME DATASET FOR ALL!!
list_of_monomial_identifiers_numbers = np.array([list(monomial_identifiers) for monomial_identifiers in list_of_monomial_identifiers]).astype(float)


loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

#X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))


seed_in_inet_training = False

loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))

seed_in_inet_training = True



In [ ]:
if use_gpu:
    from numba import cuda 
    device = cuda.get_current_device()
    device.reset()

In [ ]:
def tree_optimization(X_for_optim, y_lambda, lr=0.5, early_stopping=10, max_iter=5, worst_acceptable=0.1):
    
    def optim_func():
        X = tf.convert_to_tensor(X_for_optim, dtype=tf.float32)#[:10000,:]
        y = tf.convert_to_tensor(y_lambda, dtype=tf.float32)

        
        # HIER FV BERECHNUNG VON INET LOSS CODE?
        
        #y_input wird von optimizer übergeben über var_list
        y_pred = tf.sigmoid(y_input)

        
        feats_pred = exaggerated_softmax(tf.cast(tf.reshape(y_pred[:,:algebra.decision_nodes*algebra.d],[y_pred[:,:algebra.decision_nodes*algebra.d].shape[0],algebra.decision_nodes,algebra.d]),tf.float32))
        splits_pred = tf.cast(y_pred[:,algebra.decision_nodes*algebra.d:algebra.decision_nodes*algebra.d+algebra.decision_nodes],tf.float32)
        classes_pred = tf.cast(y_pred[:,algebra.decision_nodes*algebra.d+algebra.decision_nodes:],tf.float32)    

        

        node_probabilities = tf.vectorized_map(lambda fs: tf.math.divide(1.0,tf.math.add(1.0, tf.keras.backend.exp(tf.math.multiply(-2e1,tf.subtract(tf.tensordot(X,tf.transpose(fs[0]), axes=1),fs[1]))))), (feats_pred,splits_pred))
        leaf_probabilities = eval(algebra.tf_leaf_proba_evaluate)#tf_get_leaf(node_probabilities)
        preds = tf.einsum("ijk,ik->ij", leaf_probabilities, classes_pred)

        if inet_loss_parameters["type"]=="mae":
            result = tf.math.reduce_mean(tf.keras.backend.abs(tf.math.subtract(y, preds)),axis=1)
        elif inet_loss_parameters["type"]=="cross_entropy":
            result = tf.keras.losses.Binary_crossentropy()(y, preds)

        return tf.math.reduce_mean(result)+inet_loss_parameters["regularization_term"]*tf.math.reduce_std(result)

    
    if int(tf.__version__[0]) >= 2:
        tf.random.set_seed(RANDOM_SEED)
    else:
        tf.set_random_seed(RANDOM_SEED)     
    
    global_best = 1.0
    num_iter = 0

    while global_best>worst_acceptable and num_iter<max_iter:
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        
        #HIER RANDOM INITIALWERT FÜR PREDICTION
        y_input = tf.Variable(tf.random.uniform([1,algebra.d*algebra.decision_nodes+algebra.decision_nodes+algebra.leaf_nodes]))

        early_stop = 0
        current_best = 1.0

        while early_stop<early_stopping:
            opt.minimize(optim_func, var_list=[y_input])
            curr_res = optim_func()

            print("Current best: {} in iteration {}, Curr_res: {}".format(np.round(current_best,5), num_iter,curr_res), end='\r')
 
            early_stop += 1
            if np.round(curr_res,3)<np.round(current_best,3):
                current_best = curr_res
                early_stop = 0
                cur_best_y_pred = tf.identity(y_input)
                
        num_iter += 1

        if current_best<global_best:
            global_best = current_best
            global_best_y_pred = tf.identity(cur_best_y_pred)

    print("Optimization terminated at {}".format(global_best))
    return global_best_y_pred#,global_best